In [8]:
%pip install tensorflow
%pip install --force-reinstall numpy==1.19.0

Note: you may need to restart the kernel to use updated packages.
     ---------------------------------------- 0.0/7.3 MB ? eta -:--:--
     ---------------------------------------- 7.3/7.3 MB 63.9 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [26 lines of output]
      Running from numpy source directory.
      <string>:460: UserWarning: Unrecognized setuptools command, proceeding with generating Cython sources and expanding templates
      Traceback (most recent call last):
        File "c:\Users\acetu\anaconda3\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
          main()
        File "c:\Users\acetu\anaconda3\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "c:\Users\acetu\anaconda3\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 149, in prepare_metadata_for_build_wheel
          return hook(metadata_directory, config_settings)
 

In [9]:
%pip install opencv-python

In [1]:

import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
import cv2
import numpy as np
from numpy.linalg import norm

# Load pre-trained VGG16 model with ImageNet weights
model = VGG16(weights='imagenet', include_top=False)

# Summary of the model architecture
# model.summary()

# Load and preprocess the image from a local file path
def preprocess_image(image_path):
    img = cv2.imread(image_path)  # Read image from the local file path
    if img is None:
      raise Exception(f"Unable to load image at path: {image_path}")
    
    img = cv2.resize(img, (224, 224))  # Resize to 224x224
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    img = preprocess_input(img)  # Normalize for the CNN
    return img

# Function to describe the difference based on the distance
def describe_difference(distance):
    if distance < 100:
      return "The images are very similar, with almost no noticeable differences."
    elif 100 <= distance < 500:
      return "The images have some noticeable differences, but they are still quite similar."
    elif 500 <= distance < 1000:
      return "The images have several differences, and they appear moderately distinct from one another."
    else:
      return "The images are very different, with significant changes in content or appearance."



def cnn_process(image_path1, image_path2) :
    
  # Preprocess the images
  image1 = preprocess_image(image_path1)
  image2 = preprocess_image(image_path2)

  # Extract features from the last convolutional layer
  features1 = model.predict(image1)
  features2 = model.predict(image2)

  # Flatten the feature vectors
  features1 = features1.flatten()
  features2 = features2.flatten()

  # Calculate Euclidean distance
  distance = norm(features1 - features2)
  # print(f'Euclidean Distance between the images: {distance}')
  print(f"{distance}")
  with open("output/txt/output.txt", "a") as file:
    file.write(str(distance) + "\n")  # Add a newline after each result

  # Get a sentence describing the difference
  # description = describe_difference(distance)
  # print(description)





In [3]:
import cv2
import os

def extract_and_save_scenes(video_path, output_dir):
    """
    Extracts one frame per second from the provided video, saves them as image files locally,
    and returns scenes as a dictionary.

    Parameters:
    video_path (string): Path of the video to be processed.
    output_dir (string): Directory where the extracted frames will be saved.

    Returns:
    list of dict: Each dictionary corresponds to a scene, containing:
        - index (int): Scene index
        - img_file (string): File name of the saved image
    """
    
    if not os.path.exists(video_path):
        print(f"{video_path}: File not found.")
        return []

    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Open the video file
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    if fps <= 0:
        print(f"Error: Could not retrieve FPS from {video_path}")
        return []

    scenes = []
    index = 0
    success, frame = cap.read()
    count = 0

    while success:
        # Capture frame every second (approximately)
        if count % int(fps) == 0:
            # Convert the input image to gray scale
            # frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            # Define the file name for the saved image
            img_file = os.path.join(output_dir, f"scene_{index}.jpg")
            
            # Save the frame as an image file
            cv2.imwrite(img_file, frame)
            
            # Append scene info to the list
            scenes.append({
                "index": index,
                "img_file": img_file
            })
            
            index += 1
        
        success, frame = cap.read()
        count += 1

    cap.release()
    
    return scenes

In [4]:
import os

def compare_scenes_in_reverse(scenes, cnn_process):
    """
    Compares all scenes in reverse order using the cnn_process function.

    Parameters:
    scenes (list of dict): List of dictionaries where each dict contains:
        - index (int): Scene index
        - img_file (string): File name of the saved image
    cnn_process (function): Function to compare two images.

    Returns:
    None
    """

    # Compare images in reverse order
    for i in range(len(scenes) - 1, 0, -1):
        image_path1 = scenes[i]["img_file"]
        image_path2 = scenes[i - 1]["img_file"]
        
        # print(f"Comparing Scene {scenes[i]['index']} and Scene {scenes[i-1]['index']}:")
        # print(f"{scenes[i]['index']}-{scenes[i-1]['index']}:")
        cnn_process(image_path1, image_path2)
        # print("-" * 50)  # Separator for readability

# Example Usage:
# Assuming 'scenes' is the list returned by extract_and_save_scenes() function
# and cnn_process() is the function to compare two images.
# scenes = extract_and_save_scenes("path_to_video.mp4", "output_directory")
# compare_scenes_in_reverse(scenes, cnn_process)


In [5]:
video_path = "data/videos/test9_cs412.mp4"
output_dir = "out_directory1"
scenes = extract_and_save_scenes(video_path, output_dir)
print(f"Extracted and saved {len(scenes)} scenes.")
compare_scenes_in_reverse(scenes, cnn_process)

Extracted and saved 635 scenes.
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 784ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 253ms/step
21.074235916137695


FileNotFoundError: [Errno 2] No such file or directory: 'output/txt/output.txt'